In [ ]:
#NOTE:ALL THE STEPS ARE PERFORMED INSIDE THE RESEARCH DIRECTORY
#FOR TESTING ON OTHER CSV FILE UPLOAD TEST.CSV FILE IN RESEARCH AND IMAGE DATA IN RESEARCH->TEST_IMAGES DIRECTORY AND RUN THE CELLS 
#get data from following link before performing below steps 
#---->: https://drive.google.com/open?id=1Ut0tNBSDv1k7HRYuoHQykyMKp2oE8x7Z

In [ ]:
#Training and generation of files

In [ ]:
#Before training a model, check the present working directory and the path should be in research directory 
#Run the below step
!python setup.py install

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#For generating TF records for training data
!python generate_tfrecord.py --csv_input=data/train_labels.csv --output_path=data/train.record --image_directory=images/train

In [ ]:
#For generating TF records for test data
!python generate_tfrecord.py --csv_input=data/test_labels.csv --output_path=data/test.record --image_directory=images/test

In [ ]:
#Start training model by using below command
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_resnet101_coco_11_06_2017.config

In [ ]:
#The checkpoints will be created in the training directory after executing the above cell
#Consider the last checkpoint which is created in the training directory
#Then start testing 
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_resnet101_coco_11_06_2017.config --trained_checkpoint_prefix training/model.ckpt-2000 --output_directory resnet_output 
#Here model.ckpt-2000 is the checkpoint which is created in the training directory after training the model.

In [ ]:
#after generating res101 directory run the below commands


import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import ops as utils_ops

#if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  #raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


In [0]:
# This is needed to display the images.
%matplotlib inline

In [0]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [0]:
#import res101 folder and perform following operations
MODEL_NAME = 'res101'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('res101', 'object-detection.pbtxt')
NUM_CLASSES = 17

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
import pandas as pd
#here test.csv containiings data to be predicted
data = pd.read_csv('test.csv')
image_list = data['image_name']

#HERE WE NEED IMAGE DATA IN TEST_IMAGES BEFORE PREDICTION 
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, '{}'.format(i)) for i in image_list]

#print(TEST_IMAGE_PATHS) 
IMAGE_SIZE = (8.9, 6.7)


In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        print(real_num_detection)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
x1 = list()
x2 = list()
y1 = list()
y2 = list()
#i = 1
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  width, height = image.size
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  #print(image_np)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  boxes = (output_dict['detection_boxes'])
  #print(i)
  #np.squeeze(boxes)
  ymin = boxes[0][0]*height
  xmin = boxes[0][1]*width
  ymax = boxes[0][2]*height
  xmax = boxes[0][3]*width
  #print(xmin, xmax, ymin, ymax)
  x1.append(xmin)
  x2.append(xmax)
  y1.append(ymin)
  y2.append(ymax)
  #i += 1
  # Visualization of the results of a detection.
  '''vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)'''

In [0]:
data['x1'] = x1
data['x2'] = x2
data['y1'] = y1
data['y2'] = y2

In [0]:
data.to_csv('tested.csv', index = False, encoding = 'utf8')



##FINALLY THE DATA FILE ''TESTED.CSV ID DOWNLOADED WITH CONTAINS THE VALUES OF X1, X2, Y1, Y2